In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import spark_partition_id, col, aggregate, sum,avg,max,count
from pathlib import Path

spark = SparkSession\
        .builder\
        .appName('partition count')\
        .getOrCreate()

# Set Path
file_path = Path().cwd() / 'Datasets' / 'BankChurners.csv'
file_path = str(file_path)
file_path

In [36]:
df = spark.read.option('header', 'True').option('inferSchema', 'True').csv(file_path)
print(df.count())
print(df.rdd.getNumPartitions())

df = df.repartition(10).alias('df')
df.show(2, False)

10127
1
+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|CLIENTNUM|Attrition_Flag   |Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+-

In [37]:
# Distinct partition count
df = df.withColumn('partition_id', spark_partition_id())

In [46]:
df.groupBy("Months_on_book") \
    .agg(sum("partition_id").alias("sum_Customer_Age"))

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
df.groupBy("partition_id") \
    .agg(sum("Customer_Age").alias("sum_Customer_Age"), \
         avg("salary").alias("avg_salary"), \
         sum("bonus").alias("sum_bonus"), \
         max("bonus").alias("max_bonus") ) \
    .show(truncate=False)